In [21]:
from datasets import Dataset

questions = ["What did the president say about Justice Breyer?", 
             "What did the president say about Intel's CEO?",
             "What did the president say about gun violence?",
            ]
ground_truths = [["The president said that Justice Breyer has dedicated his life to serve the country and thanked him for his service."],
                ["The president said that Pat Gelsinger is ready to increase Intel's investment to $100 billion."],
                ["The president asked Congress to pass proven measures to reduce gun violence."]]
answers = ["alkmdsa;lk", "jhgfdsgdfgh", "zxcvbnm,./"]
contexts = [["aklsjdal"],["klasmlda"],["klsmdla"]]

In [22]:
data = {
    "question":  ["What is the capital of France?"],
    "answer": ["Paris"],
    # "contexts": [["Paris is the capital of France."]],
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [23]:
dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 1
})

In [24]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset, 
    metrics=[
        # context_precision,
        # context_recall,
        faithfulness,
        # answer_relevancy,
    ],
)

ValueError: The metric [faithfulness] that that is used requires the following additional columns ['contexts'] to be present in the dataset. 

In [27]:
result["faithfulness"]

1.0

In [32]:
import asyncio
from ragas.metrics._faithfulness import Faithfulness

async def main():
    faithfulness = Faithfulness()
    results = await faithfulness.score(dataset)

# Check if an event loop is already running
if asyncio.get_event_loop().is_running():
    # If running, create a new task and run it on the existing event loop
    task = asyncio.ensure_future(main())
    asyncio.get_event_loop().run_until_complete(task)
else:
    # If no event loop is running, use asyncio.run()
    asyncio.run(main())


c:\Users\polasani rohit\AppData\Local\Programs\Python\Python311\Lib\ast.py:50: RuntimeWarning: coroutine 'main' was never awaited
  return compile(source, filename, mode, flags,


RuntimeError: This event loop is already running

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-base-en')
question_vec = embeddings.embed_query("question")
answer_vec = embeddings.embed_query("answer")


c:\Users\polasani rohit\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
.gitattributes: 100%|██████████| 1.52k/1.52k [00:00<?, ?B/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 185kB/s]
README.md: 100%|██████████| 90.1k/90.1k [00:00<00:00, 3.22MB/s]
config_sentence_transformers.json: 100%|██████████| 124/124 [00:00<?, ?B/s] 
model.safetensors: 100%|██████████| 438M/438M [05:51<00:00, 1.25MB/s] 
pytorch_model.bin: 100%|██████████| 438M/438M [05:51<00:00, 1.25MB/s] 
sentence_bert_config.json: 100%|██████████| 52.0/52.0 [00:00<?, ?B/s]
special_tokens_map.json: 100%|██████████| 125/125 [00:00<?, ?B/s] 
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 1.21MB/s]
tokenizer_config.json: 100%|██████████| 366/366 [00:00<?, ?B/s] 
vocab.txt: 100%|██████████| 2

In [5]:
question_vec

768

In [6]:
len(answer_vec)

768

In [8]:
from transformers import AutoModel, AutoTokenizer
import pickle

# Model name
model_name = "BAAI/bge-base-en"

# Load the model and tokenizer
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Save the model and tokenizer using pickle
with open("model_tokenizer.pkl", "wb") as file:
    pickle.dump((model, tokenizer), file)


model.safetensors: 100%|██████████| 438M/438M [06:26<00:00, 1.13MB/s] 
tokenizer_config.json: 100%|██████████| 366/366 [00:00<?, ?B/s] 
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 572kB/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 1.56MB/s]
special_tokens_map.json: 100%|██████████| 125/125 [00:00<00:00, 136kB/s]


In [18]:
pickled_model = pickle.load(open('model_tokenizer.pkl', 'rb'))

EOFError: Ran out of input

In [16]:
pickled_model.

(BertModel(
   (embeddings): BertEmbeddings(
     (word_embeddings): Embedding(30522, 768, padding_idx=0)
     (position_embeddings): Embedding(512, 768)
     (token_type_embeddings): Embedding(2, 768)
     (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
     (dropout): Dropout(p=0.1, inplace=False)
   )
   (encoder): BertEncoder(
     (layer): ModuleList(
       (0-11): 12 x BertLayer(
         (attention): BertAttention(
           (self): BertSelfAttention(
             (query): Linear(in_features=768, out_features=768, bias=True)
             (key): Linear(in_features=768, out_features=768, bias=True)
             (value): Linear(in_features=768, out_features=768, bias=True)
             (dropout): Dropout(p=0.1, inplace=False)
           )
           (output): BertSelfOutput(
             (dense): Linear(in_features=768, out_features=768, bias=True)
             (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
             (dropout): Dropout(p=

In [26]:
from ragas.metrics._answer_relevance import AnswerRelevancy

In [13]:
from ragas.llms.prompt import Prompt

QUESTION_GEN = Prompt(
    name="question_generation",
    instruction="""Generate a question for the given answer and Identify if answer is noncommittal. Give noncommittal as 1 if the answer is noncommittal and 0 if the answer is committal. A noncommittal answer is one that is evasive, vague, or ambiguous. For example, "I don't know" or "I'm not sure" are noncommittal answers.""",
    examples=[
        {
            "answer": """Albert Einstein was born in Germany.""",
            "context": """Albert Einstein was a German-born theoretical physicist who is widely held to be one of the greatest and most influential scientists of all time""",
            "output": {
                "question": "Where was Albert Einstein born?",
                "noncommittal": 0,
            },
        },
        {
            "answer": """It can change its skin color based on the temperature of its environment.""",
            "context": """A recent scientific study has discovered a new species of frog in the Amazon rainforest that has the unique ability to change its skin color based on the temperature of its environment.""",
            "output": {
                "question": "What unique ability does the newly discovered species of frog have?",
                "noncommittal": 0,
            },
        },
        {
            "answer": """Everest""",
            "context": """The tallest mountain on Earth, measured from sea level, is a renowned peak located in the Himalayas.""",
            "output": {
                "question": "What is the tallest mountain on Earth?",
                "noncommittal": 0,
            },
        },
        {
            "answer": """I don't know about the  groundbreaking feature of the smartphone invented in 2023 as am unaware of information beyond 2022. """,
            "context": """In 2023, a groundbreaking invention was announced: a smartphone with a battery life of one month, revolutionizing the way people use mobile technology.""",
            "output": {
                "question": "What was the groundbreaking feature of the smartphone invented in 2023?",
                "noncommittal": 1,
            },
        },
    ],
    input_keys=["answer", "context"],
    output_key="output",
    output_type="json",
)

In [14]:
type(QUESTION_GEN)

ragas.llms.prompt.Prompt

In [15]:
prompt1 = QUESTION_GEN.format(answer="newdelhi", context="Paris is the capital of France.new delhi is also a capital")

In [16]:
prompt = QUESTION_GEN.to_string()

In [17]:
print(type(prompt))

<class 'str'>


In [18]:
from langchain.llms import OpenAI

In [19]:
from ragas.llms.base import LangchainLLMWrapper
class  LLMSRagAsm(LangchainLLMWrapper):
    def __init__(self,prompt):
        self.prompt = prompt

    def gen(self):
        ans = LangchainLLMWrapper(langchain_llm=OpenAI())

        return ans.generate_text(prompt=self.prompt,n=3)

obj = LLMSRagAsm(prompt=prompt1)


In [21]:
obj.gen().generations[0][0].text

'[{"question": "What is the capital of France?", "noncommittal": 0}, {"question": "What is the capital of India?", "noncommittal": 1}, {"question": "Can you name a city that is a capital?", "noncommittal": 1}]'